# VIA标记转换为Unet掩码图

In [1]:
import tensorflow as tf
print(tf.test.is_gpu_available())

True


In [2]:
import os
import sys
import shutil
import random

import pandas as pd
import cv2

# 添加路径，以能正常导入mbsh、trainer
sys.path.insert(0, 'trainer')

from mbsh.core.images import fetch_all_files, read_img_file, resize_image
from mbsh.core.mask_rcnn import balloon

import glob

2022-05-06 16:52:41,299 - INFO - __init__.py - init_logger - 58 - ini  logger file D:\projects\znyx-trainer\logs\kernel-c3e8aa57-3a7f-4362-a70a-e142d9a9971d.log


use temp dir:d:\znyx\temp


Using TensorFlow backend.
2022-05-06 16:52:41,554 - DEBUG - __init__.py - wrapper - 241 - (private) matplotlib data path: C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\matplotlib\mpl-data
2022-05-06 16:52:41,556 - DEBUG - __init__.py - wrapper - 241 - matplotlib data path: C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\matplotlib\mpl-data
2022-05-06 16:52:41,561 - DEBUG - __init__.py - wrapper - 241 - CONFIGDIR=C:\Users\Administrator\.matplotlib
2022-05-06 16:52:41,563 - DEBUG - __init__.py - <module> - 1480 - matplotlib version 3.3.4
2022-05-06 16:52:41,564 - DEBUG - __init__.py - <module> - 1481 - interactive is False
2022-05-06 16:52:41,565 - DEBUG - __init__.py - <module> - 1482 - platform is win32
2022-05-06 16:52:41,565 - DEBUG - __init__.py - <module> - 1483 - loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs',

2022-05-06 16:52:41,623 - DEBUG - __init__.py - wrapper - 241 - CACHEDIR=C:\Users\Administrator\.matplotlib
2022-05-06 16:52:41,628 - DEBUG - font_manager.py - <module> - 1436 - Using fontManager instance from C:\Users\Administrator\.matplotlib\fontlist-v330.json
2022-05-06 16:52:41,767 - DEBUG - pyplot.py - switch_backend - 290 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2022-05-06 16:52:41,771 - DEBUG - pyplot.py - switch_backend - 290 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


2022-05-06 16:52:42,632 - WARNING - module_wrapper.py - _tfmw_add_deprecation_warning - 139 - From trainer\mbsh\core\yolo3\model.py:17: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



2022-05-06 16:52:42,637 - WARNING - module_wrapper.py - _tfmw_add_deprecation_warning - 139 - From trainer\mbsh\core\yolo3\model.py:17: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.



2022-05-06 16:52:42,642 - WARNING - module_wrapper.py - _tfmw_add_deprecation_warning - 139 - From trainer\mbsh\core\yolo3\model.py:18: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



### 配置

In [3]:
# 图片文件目录，包括VIA标记的csv文件
input_path = r'D:\项目资料\胆结石\胆结石分割模型\胆结石图片'
csv_file_name = glob.glob(input_path+'\\*.csv')
if len(csv_file_name) != 1:
    csv_file_name = None
else:
    csv_file_name = os.path.basename(csv_file_name[0])
    print(csv_file_name)
# csv_file_name = r'I:\郑碧清暂存\静脉曲张\cy wdjmqz\rm.csv'

# 掩码图存放目录
mask_path = input_path + '_mask0317'

# resize后原图、掩码图存放目录
result_path = input_path + '_result'

# 需要把图片resize的尺寸，None表示不做尺寸变化
img_size = None

# 验证集占比
valid_split_ratio = 0.2

# 测试集占比
test_split_ratio = 0.2

# resize后原图存放子目录，分离测试集后即为训练集目录
image_result_path = os.path.join(result_path, 'images')
# resize后lebel掩码图存放子目录，分离测试集后即为训练集掩码子目录
label_result_path = os.path.join(result_path, 'labels')

# 验证集原图、掩码图子目录
valid_path = os.path.join(result_path, 'valid_images')
valid_label_path = os.path.join(result_path, 'valid_labels')


# 测试集原图、掩码图子目录
test_path = os.path.join(result_path, 'test_images')
test_label_path = os.path.join(result_path, 'test_labels')

胆结石分割标记.csv


### 删除未做人工标记的图片

In [5]:
# 分别从csv文件和文件目录读取文件列表


# csv全路径
csv_path = os.path.join(input_path, csv_file_name)

# 从csv文件读取文件名列表，存放到csv_files，文件名在csv文件第一列
# _csv = pd.read_csv(csv_path, encoding='utf-8', usecols=['filename'], engine='python')
_csv = pd.read_csv(csv_path, usecols=['filename'])
_csv_files = _csv.values.tolist()
csv_files = []
for i in range(len(_csv_files)):
    csv_files.append(_csv_files[i][0])

    
# 读取图片目录的所有非csv文件文件名
files = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f)) and f.split('.')[-1] != 'csv']

In [6]:
# 对目录中文件，如果未在csv文件中标记，则删除
del_num = 0

for file in files:
    if file not in csv_files:
        del_num += 1
        print(file)
        abs_f = os.path.join(input_path, file)
#         if os.path.exists(abs_f):
#             os.remove(abs_f)
        
print(del_num, "个文件未标记。")

0 个文件未标记。


### 根据VIA的csv文件生成掩码图

In [7]:
def make_dir(dir):
    # 结果生成路径如果存在，则删除
    if os.path.exists(dir):
        shutil.rmtree(dir)

    os.makedirs(dir)

In [8]:
# 建立掩码图存放目录
make_dir(mask_path)

In [9]:
# Load dataset
# Get the dataset from the releases page
# https://github.com/matterport/Mask_RCNN/releases
dataset = balloon.BalloonDataset()
dataset.load_data_from_csv(input_path, csv_file_name)

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))#字符串格式化
print("Class Count: {}".format(dataset.num_classes))

D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000002549.0007.1480407196.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [351, 347, 341, 338, 332, 328, 323, 317, 312, 309, 309, 309, 311, 315, 320, 326, 331, 337, 345, 352, 356, 362, 366, 370, 372, 370, 366, 363, 362], 'all_points_y': [121, 122, 122, 121, 118, 112, 110, 109, 110, 113, 119, 122, 127, 131, 136, 138, 144, 148, 148, 148, 148, 148, 146, 142, 133, 128, 125, 123, 122]} tmp_dict 01.0000000002549.0007.1480407196.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000002549.0008.1480407209.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [359, 352, 345, 338, 331, 327, 321, 315, 311, 308, 309, 312, 320, 326, 331, 340, 348, 354, 359, 364, 367, 369, 372, 371, 365], 'all_points_y': [123, 122, 124, 121, 118, 115, 110, 109, 111, 115, 121, 128, 136, 143, 146, 147, 146, 148, 148, 147, 142, 141, 134, 127, 123]} tmp_dict 01.0000000002549.0008.1480407209.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000002549.0009.1480407301.jpg <class 'str

{'name': 'polygon', 'all_points_x': [287, 279, 277, 270, 265, 264, 266, 270, 275, 281, 285, 288], 'all_points_y': [131, 123, 122, 123, 129, 133, 139, 150, 153, 151, 148, 142]} tmp_dict 01.0000000008371.0041.1486454636.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000008371.0042.1486454638.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [262, 264, 274, 281, 289, 294, 288, 283, 275, 268, 266], 'all_points_y': [131, 138, 146, 151, 151, 146, 129, 120, 118, 121, 123]} tmp_dict 01.0000000008371.0042.1486454638.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000008371.0043.1486454644.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [262, 264, 269, 279, 285, 291, 291, 288, 281, 276, 273, 268], 'all_points_y': [129, 135, 144, 148, 150, 146, 139, 131, 120, 119, 119, 121]} tmp_dict 01.0000000008371.0043.1486454644.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000010026.0013.1487662977.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [333, 328, 323, 319, 319, 321, 323,

{'name': 'polygon', 'all_points_x': [42, 39, 35, 28, 20, 15, 16, 19, 25, 36, 43, 47, 47], 'all_points_y': [182, 173, 166, 166, 166, 172, 185, 191, 195, 198, 199, 196, 190]} tmp_dict 01.0000000045296.0045.1515398117.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000045296.0046.1515398133.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [59, 54, 49, 41, 37, 31, 26, 26, 26, 29, 33, 38, 45, 51, 59, 64, 64, 64], 'all_points_y': [189, 180, 174, 174, 174, 175, 181, 186, 192, 200, 204, 205, 206, 206, 206, 202, 198, 197]} tmp_dict 01.0000000045296.0046.1515398133.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000045296.0047.1515398144.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [49, 42, 34, 29, 27, 25, 25, 28, 37, 41, 49, 55, 59, 59, 57, 56], 'all_points_y': [184, 180, 176, 178, 186, 190, 197, 204, 206, 208, 208, 206, 204, 198, 194, 190]} tmp_dict 01.0000000045296.0047.1515398144.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000052833.0015.1522133381.jpg <class 'str'> <class 'str'

{'name': 'polygon', 'all_points_x': [332, 318, 313, 309, 304, 297, 295, 296, 304, 317, 323, 330, 340, 345, 346, 346, 346, 340, 339, 338], 'all_points_y': [151, 150, 151, 155, 160, 168, 174, 181, 192, 206, 209, 209, 206, 204, 200, 191, 182, 172, 165, 163]} tmp_dict 01.0000000088423.0063.1544503032.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000088691.0054.1544587988.jpg <class 'str'> <class 'str'>
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000088691.0055.1544587988.jpg <class 'str'> <class 'str'>
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000089973.0011.1545361497.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [444, 444, 441, 434, 425, 421, 419, 414, 414, 417, 420, 423, 427, 431, 435, 438, 442], 'all_points_y': [180, 174, 168, 163, 162, 162, 164, 173, 178, 182, 183, 181, 176, 174, 178, 178, 178]} tmp_dict 01.0000000089973.0011.1545361497.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000089973.0012.1545361498.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [450, 444, 438, 429, 424, 

{'name': 'polygon', 'all_points_x': [548, 539, 532, 510, 492, 496, 510, 530, 551, 563, 563, 560, 551], 'all_points_y': [316, 328, 333, 332, 311, 289, 275, 273, 278, 285, 292, 301, 309]} tmp_dict 01.0000000094454.0028.1548382882.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000094454.0029.1548382903.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [555, 555, 567, 568, 558, 532, 513, 498, 489, 487, 487, 510, 518, 539, 549, 555], 'all_points_y': [311, 309, 301, 285, 276, 264, 264, 273, 283, 297, 313, 323, 332, 339, 325, 318]} tmp_dict 01.0000000094454.0029.1548382903.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000095754.0026.1549939083.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [528, 537, 533, 520, 509, 506, 504, 502, 502, 506, 511, 523, 526, 526], 'all_points_y': [161, 152, 142, 135, 142, 147, 155, 162, 174, 186, 186, 186, 173, 169]} tmp_dict 01.0000000095754.0026.1549939083.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000095754.0027.1549939086.jpg <class 'str'> <clas

...

{'name': 'polygon', 'all_points_x': [939, 931, 923, 911, 903, 903, 907, 919, 933, 931, 933], 'all_points_y': [310, 298, 298, 304, 318, 330, 340, 340, 332, 334, 332]} tmp_dict 01.0000000216300.0023.1636518493.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000216300.0024.1636518497.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [923, 897, 881, 879, 885, 895, 903, 919, 927], 'all_points_y': [432, 420, 426, 438, 446, 450, 454, 452, 446]} tmp_dict 01.0000000216300.0024.1636518497.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000216300.0025.1636518516.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [378, 372, 358, 342, 337, 330, 320, 316, 321, 337, 368, 377, 378], 'all_points_y': [325, 315, 308, 309, 318, 328, 339, 346, 366, 368, 354, 346, 339]} tmp_dict 01.0000000216300.0025.1636518516.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000216300.0026.1636518520.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [377, 359, 347, 340, 334, 320, 318, 323, 337, 351, 358,

D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000219826.0104.1638419709.jpg <class 'str'> <class 'str'>
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000219826.0105.1638419732.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1275, 1269, 1257, 1257, 1265, 1279, 1279], 'all_points_y': [233, 239, 237, 225, 217, 219, 235]} tmp_dict 01.0000000219826.0105.1638419732.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000219826.0115.1638419863.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1136, 1112, 1104, 1100, 1100, 1116, 1126, 1136, 1136], 'all_points_y': [217, 207, 217, 225, 231, 227, 225, 231, 225]} tmp_dict 01.0000000219826.0115.1638419863.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000219826.0118.1638419920.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1128, 1114, 1094, 1092, 1104, 1116, 1126, 1132], 'all_points_y': [233, 231, 233, 243, 249, 249, 251, 249]} tmp_dict 01.0000000219826.0118.1638419920.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000219826.0119.1638419921.jpg <cl

{'name': 'polygon', 'all_points_x': [1363, 1353, 1333, 1321, 1315, 1315, 1321, 1329, 1339, 1345, 1361, 1373, 1383, 1379, 1363, 1353, 1343, 1341, 1353, 1355, 1359], 'all_points_y': [189, 197, 207, 207, 221, 231, 245, 249, 259, 269, 275, 277, 263, 249, 247, 245, 233, 221, 213, 209, 205]} tmp_dict 01.0000000225935.0077.1642047288.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000225935.0078.1642047289.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1361, 1351, 1343, 1339, 1327, 1319, 1315, 1317, 1331, 1337, 1345, 1353, 1363, 1365, 1357, 1345], 'all_points_y': [203, 189, 193, 205, 209, 215, 229, 239, 247, 257, 263, 265, 265, 259, 249, 219]} tmp_dict 01.0000000225935.0078.1642047289.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000227356.0109.1643001361.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1226, 1218, 1204, 1198, 1192, 1182, 1200, 1222, 1222], 'all_points_y': [354, 358, 362, 364, 370, 366, 348, 346, 346]} tmp_dict 01.0000000227356.0109.1643001361.jpg
D:\项目资

{'name': 'polygon', 'all_points_x': [1136, 1122, 1102, 1084, 1076, 1080, 1084, 1102, 1122, 1132, 1148, 1148, 1148, 1142], 'all_points_y': [225, 245, 247, 253, 259, 265, 269, 275, 271, 269, 249, 237, 231, 221]} tmp_dict 01.0000000231924.0094.1646362119.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000231924.0095.1646362120.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1138, 1126, 1112, 1104, 1096, 1080, 1068, 1070, 1086, 1120, 1132, 1140, 1140, 1140], 'all_points_y': [233, 235, 249, 251, 253, 259, 271, 277, 281, 265, 255, 243, 241, 239]} tmp_dict 01.0000000231924.0095.1646362120.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000231924.0096.1646362120.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1144, 1128, 1120, 1108, 1088, 1084, 1090, 1116, 1140, 1150, 1150, 1150], 'all_points_y': [225, 231, 245, 251, 257, 269, 275, 281, 263, 249, 231, 225]} tmp_dict 01.0000000231924.0096.1646362120.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000231924.0097.1646362121.jpg <class 's

D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000232687.0048.1646884380.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1172, 1162, 1146, 1134, 1116, 1104, 1112, 1120, 1132, 1138, 1152, 1174, 1182], 'all_points_y': [227, 209, 205, 205, 209, 231, 243, 241, 231, 227, 225, 245, 243]} tmp_dict 01.0000000232687.0048.1646884380.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000232687.0049.1646884403.jpg <class 'str'> <class 'str'>
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000232687.0052.1646884436.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1154, 1142, 1124, 1108, 1102, 1122, 1156, 1164, 1168, 1166], 'all_points_y': [285, 273, 269, 277, 292, 306, 310, 302, 292, 292]} tmp_dict 01.0000000232687.0052.1646884436.jpg
D:\项目资料\胆结石\胆结石分割模型\胆结石图片 01.0000000232687.0053.1646884455.jpg <class 'str'> <class 'str'>
{'name': 'polygon', 'all_points_x': [1174, 1166, 1150, 1132, 1116, 1102, 1094, 1092, 1108, 1112, 1128, 1140, 1152, 1168, 1176], 'all_points_y': [267, 257, 249, 245, 245, 249, 263,

In [10]:
for i in dataset.image_ids:
    print(i, dataset)
#     try:
    mask = dataset.load_mask(i)
#     except:
#         print('error')
#         continue
    image = dataset.load_image(i)
    print(image.shape, mask[0].shape, dataset.image_info[i]['path'])
    splash = balloon.get_black_white_mask_img(image, mask[0])
    
    file_name = dataset.image_info[i]['path']
    fpath,fname = os.path.split(file_name)
    new_fn = os.path.join(mask_path, fname)
    
#     cv2.imwrite(new_fn, splash)
    cv2.imencode('.jpg', splash)[1].tofile(new_fn)
#     if  i >= 20:
#         break

0 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
29 len(p[])
(576, 720, 1) {'id': '01.0000000002549.0007.1480407196.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000002549.0007.1480407196.jpg', 'width': 720, 'height': 576, 'polygons': [{'name': 'polygon', 'all_points_x': [351, 347, 341, 338, 332, 328, 323, 317, 312, 309, 309, 309, 311, 315, 320, 326, 331, 337, 345, 352, 356, 362, 366, 370, 372, 370, 366, 363, 362], 'all_points_y': [121, 122, 122, 121, 118, 112, 110, 109, 110, 113, 119, 122, 127, 131, 136, 138, 144, 148, 148, 148, 148, 148, 146, 142, 133, 128, 125, 123, 122]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000002549.0007.1480407196.jpg 3 (576, 720, 3) ppppppppppppp
(576, 720, 3) (576, 720, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000002549.0007.1480407196.jpg
1 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
25 len(p[])
(576, 720, 1) {'id': '01.0000000002549.0008.1480407209.jpg', 'source': '

42 len(p[])
(1050, 1155, 1) {'id': '01.0000000005583.0005.1483158088.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000005583.0005.1483158088.jpg', 'width': 1155, 'height': 1050, 'polygons': [{'name': 'polygon', 'all_points_x': [635, 621, 602, 580, 561, 534, 526, 522, 513, 509, 503, 501, 497, 489, 493, 509, 524, 532, 534, 540, 544, 551, 559, 575, 584, 592, 598, 600, 606, 615, 627, 637, 656, 664, 671, 687, 696, 696, 695, 681, 656, 644], 'all_points_y': [253, 259, 259, 263, 267, 284, 298, 302, 313, 333, 352, 368, 379, 389, 400, 402, 399, 383, 356, 342, 335, 335, 348, 356, 356, 350, 335, 317, 310, 304, 306, 313, 313, 319, 321, 321, 313, 302, 294, 282, 257, 257]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000005583.0005.1483158088.jpg 3 (1050, 1155, 3) ppppppppppppp
(1050, 1155, 3) (1050, 1155, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000005583.0005.1483158088.jpg
11 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
30 len(p[])
(1050, 

20 len(p[])
(576, 720, 1) {'id': '01.0000000010927.0011.1488333062.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000010927.0011.1488333062.jpg', 'width': 720, 'height': 576, 'polygons': [{'name': 'polygon', 'all_points_x': [357, 355, 347, 343, 338, 336, 334, 332, 334, 337, 340, 347, 353, 358, 364, 368, 371, 371, 366, 364], 'all_points_y': [145, 147, 148, 148, 148, 148, 151, 155, 157, 160, 161, 163, 168, 168, 166, 163, 158, 152, 148, 146]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000010927.0011.1488333062.jpg 3 (576, 720, 3) ppppppppppppp
(576, 720, 3) (576, 720, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000010927.0011.1488333062.jpg
32 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
16 len(p[])
(576, 720, 1) {'id': '01.0000000012754.0014.1489649710.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000012754.0014.1489649710.jpg', 'width': 720, 'height': 576, 'polygons': [{'name': 'polygon', 'all_points_x':

11 len(p[])
(551, 537, 1) {'id': '01.0000000032204.0014.1504668760.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000032204.0014.1504668760.jpg', 'width': 537, 'height': 551, 'polygons': [{'name': 'polygon', 'all_points_x': [109, 107, 105, 100, 96, 95, 97, 103, 105, 106, 107], 'all_points_y': [152, 148, 145, 144, 148, 151, 156, 156, 156, 157, 157]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000032204.0014.1504668760.jpg 3 (551, 537, 3) ppppppppppppp
(551, 537, 3) (551, 537, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000032204.0014.1504668760.jpg
51 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
16 len(p[])
(486, 629, 1) {'id': '01.0000000032204.0017.1504668996.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000032204.0017.1504668996.jpg', 'width': 629, 'height': 486, 'polygons': [{'name': 'polygon', 'all_points_x': [305, 302, 298, 295, 293, 288, 283, 278, 277, 283, 293, 298, 303, 307, 308, 308], 'all_point

18 len(p[])
(527, 436, 1) {'id': '01.0000000045296.0046.1515398133.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000045296.0046.1515398133.jpg', 'width': 436, 'height': 527, 'polygons': [{'name': 'polygon', 'all_points_x': [59, 54, 49, 41, 37, 31, 26, 26, 26, 29, 33, 38, 45, 51, 59, 64, 64, 64], 'all_points_y': [189, 180, 174, 174, 174, 175, 181, 186, 192, 200, 204, 205, 206, 206, 206, 202, 198, 197]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000045296.0046.1515398133.jpg 3 (527, 436, 3) ppppppppppppp
(527, 436, 3) (527, 436, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000045296.0046.1515398133.jpg
71 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
16 len(p[])
(341, 432, 1) {'id': '01.0000000045296.0047.1515398144.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000045296.0047.1515398144.jpg', 'width': 432, 'height': 341, 'polygons': [{'name': 'polygon', 'all_points_x': [49, 42, 34, 29, 27, 25, 25, 28, 37, 

...

1283 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
17 len(p[])
(1080, 1920, 1) {'id': '01.0000000231924.0066.1646361848.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000231924.0066.1646361848.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name': 'polygon', 'all_points_x': [1088, 1051, 1003, 983, 963, 963, 985, 999, 1029, 1053, 1076, 1088, 1100, 1106, 1106, 1106, 1100], 'all_points_y': [255, 265, 265, 269, 283, 300, 304, 304, 304, 306, 302, 298, 292, 277, 273, 269, 255]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000231924.0066.1646361848.jpg 3 (1080, 1920, 3) ppppppppppppp
(1080, 1920, 3) (1080, 1920, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000231924.0066.1646361848.jpg
1284 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
19 len(p[])
(1080, 1920, 1) {'id': '01.0000000231924.0067.1646361849.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000231924.0067.1646361849.jpg', 'w

1295 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
12 len(p[])
(1080, 1920, 1) {'id': '01.0000000231924.0096.1646362120.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000231924.0096.1646362120.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name': 'polygon', 'all_points_x': [1144, 1128, 1120, 1108, 1088, 1084, 1090, 1116, 1140, 1150, 1150, 1150], 'all_points_y': [225, 231, 245, 251, 257, 269, 275, 281, 263, 249, 231, 225]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000231924.0096.1646362120.jpg 3 (1080, 1920, 3) ppppppppppppp
(1080, 1920, 3) (1080, 1920, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000231924.0096.1646362120.jpg
1296 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
12 len(p[])
(1080, 1920, 1) {'id': '01.0000000231924.0097.1646362121.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000231924.0097.1646362121.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name':

18 len(p[])
(1080, 1920, 1) {'id': '01.0000000231924.0108.1646362216.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000231924.0108.1646362216.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name': 'polygon', 'all_points_x': [1082, 1066, 1043, 1025, 1003, 989, 991, 1003, 1005, 1015, 1029, 1035, 1035, 1041, 1058, 1064, 1072, 1078], 'all_points_y': [245, 237, 235, 235, 239, 249, 259, 269, 281, 287, 287, 283, 269, 255, 255, 257, 257, 253]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000231924.0108.1646362216.jpg 3 (1080, 1920, 3) ppppppppppppp
(1080, 1920, 3) (1080, 1920, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000231924.0108.1646362216.jpg
1308 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
18 len(p[])
(1080, 1920, 1) {'id': '01.0000000231924.0109.1646362217.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000231924.0109.1646362217.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name': 'polygon', 'al

1319 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
9 len(p[])
(1080, 1920, 1) {'id': '01.0000000232107.0087.1646621420.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000232107.0087.1646621420.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name': 'polygon', 'all_points_x': [979, 969, 955, 951, 959, 969, 977, 981, 983], 'all_points_y': [167, 157, 157, 165, 171, 179, 183, 181, 177]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000232107.0087.1646621420.jpg 3 (1080, 1920, 3) ppppppppppppp
(1080, 1920, 3) (1080, 1920, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000232107.0087.1646621420.jpg
1320 <mbsh.core.mask_rcnn.balloon.BalloonDataset object at 0x000001ED0F3197F0>
12 len(p[])
(1080, 1920, 1) {'id': '01.0000000232107.0088.1646621451.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000232107.0088.1646621451.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name': 'polygon', 'all_points_x': [1120, 1094, 10

12 len(p[])
(1080, 1920, 1) {'id': '01.0000000232687.0059.1646884486.jpg', 'source': 'region', 'path': 'D:\\项目资料\\胆结石\\胆结石分割模型\\胆结石图片\\01.0000000232687.0059.1646884486.jpg', 'width': 1920, 'height': 1080, 'polygons': [{'name': 'polygon', 'all_points_x': [1126, 1116, 1100, 1086, 1076, 1070, 1084, 1096, 1112, 1130, 1134, 1136], 'all_points_y': [336, 332, 328, 330, 334, 346, 350, 350, 362, 366, 362, 354]}]} zzzzzzzzzzzz True
D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000232687.0059.1646884486.jpg 3 (1080, 1920, 3) ppppppppppppp
(1080, 1920, 3) (1080, 1920, 1) D:\项目资料\胆结石\胆结石分割模型\胆结石图片\01.0000000232687.0059.1646884486.jpg


In [11]:
import numpy as np

In [12]:
img = cv2.imdecode(np.fromfile(r'F:\ai_data\zaoai\1204\ca\yt\IMG_01.0000001069161.24114.0067.16193400838.jpg', dtype=np.uint8), -1)
print(img.ndim, img.shape, 'ppppppppppppp')

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\ai_data\\zaoai\\1204\\ca\\yt\\IMG_01.0000001069161.24114.0067.16193400838.jpg'

In [27]:
image = cv2.imread(r'F:\ai_data\zaoai\1204\ca\yt\IMG_01.0000001069161.24114.0067.16193400838.jpg')
print(image.ndim, image.shape, 'ppppppppppppp')

3 (1032, 1173, 3) ppppppppppppp


In [21]:
image = cv2.imread(r'F:\ai_data\zaoai\1204\ca\yt\IMG_01.0000000999151.0094.14592200469.jpg')
print(image.ndim, image.shape, 'ppppppppppppp')

3 (1038, 1186, 3) ppppppppppppp


In [18]:
image = cv2.imdecode(np.fromfile(r'F:\ai_data\zaoai\1204\ca\yt\IMG_01.0000001069161.24114.0067.16193400838.jpg', dtype=np.uint8), -1)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.ndim, image.shape, 'ppppppppppppp')

3 (1173, 1032, 3) ppppppppppppp


In [19]:
image = cv2.imdecode(np.fromfile(r'F:\ai_data\zaoai\1204\ca\yt\IMG_01.0000000999151.0094.14592200469.jpg', dtype=np.uint8), -1)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.ndim, image.shape, 'ppppppppppppp')

3 (1038, 1186, 3) ppppppppppppp


### 统一图片尺寸

In [23]:
make_dir(result_path)

In [24]:
def resize_files_list(files_list, result_path):
    for img_path in files_list:
        fpath,fname = os.path.split(img_path)
        file_name = os.path.join(result_path, fname)

        img = read_img_file(img_path)
#         img = resize_image(img, img_size)
        img = cv2.resize(img, img_size)

        cv2.imencode('.jpg', img)[1].tofile(file_name)

In [25]:
# 原图
make_dir(image_result_path)

img_all_files = fetch_all_files(input_path, file_exts = ['.jpg', '.png', '.JPG'])

resize_files_list(img_all_files, image_result_path)

2021-01-05 09:50:43,732 - DEBUG - images.py - fetch_all_files - 303 - exclude file noca-re.csv,.csv
2021-01-05 09:50:43,732 - DEBUG - images.py - fetch_all_files - 315 - fetch_all_files count=218


In [26]:
# label的掩码图
make_dir(label_result_path)

label_all_files = fetch_all_files(mask_path, file_exts = ['.jpg', '.png', '.JPG'])

resize_files_list(label_all_files, label_result_path)

2021-01-05 09:50:49,347 - DEBUG - images.py - fetch_all_files - 315 - fetch_all_files count=218


### 根据比例划分训练集、测试集

In [13]:
img_result_files = fetch_all_files(image_result_path, file_exts = ['.jpg', '.png', '.JPG'])
random.shuffle(img_result_files)

make_dir(test_path)
make_dir(test_label_path)

test_num = int(test_split_ratio * len(img_result_files))

for i in range(test_num):
    shutil.move(img_result_files[i], test_path)
    
    fpath,fname=os.path.split(img_result_files[i])
    
    label_file = os.path.join(label_result_path, fname)
    shutil.move(label_file, test_label_path)

2021-01-05 09:43:43,268 - DEBUG - images.py - fetch_all_files - 315 - fetch_all_files count=213


### 根据比例划分训练集、验证集

In [27]:
img_result_files = fetch_all_files(image_result_path, file_exts = ['.jpg', '.png', '.JPG'])
random.shuffle(img_result_files)

make_dir(valid_path)
make_dir(valid_label_path)

valid_num = int(valid_split_ratio * len(img_result_files))

for i in range(valid_num):
    shutil.move(img_result_files[i], valid_path)
    
    fpath,fname=os.path.split(img_result_files[i])
    
    label_file = os.path.join(label_result_path, fname)
    shutil.move(label_file, valid_label_path)

2021-01-05 10:28:15,478 - DEBUG - images.py - fetch_all_files - 315 - fetch_all_files count=196
